In [1]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [2]:
print("Total number of charaters",len(raw_text))

Total number of charaters 20479


In [3]:
print(raw_text[:99])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


#####   Our goal is to tokkenize this 20479 characters into individual word and special characters that we can then turn into embedding for LLM training

### Example of how splitting show be done 

In [4]:
import re 


text="Hello , LLMs. Split this sentence."
result=re.split(r'(\s)',text)

print(result)

['Hello', ' ', ',', ' ', 'LLMs.', ' ', 'Split', ' ', 'this', ' ', 'sentence.']


#### whenever there whitespace it split there
####   and we also we , . as token the 

In [5]:
result=re.split(r'([,.] | \s)', text)
result

['Hello ', ', ', 'LLMs', '. ', 'Split this sentence.']

#### we dont need whitespace token (if required like for llm for python code then we include it )

In [6]:
result=[item for item in result if item.strip()]
print(result)

['Hello ', ', ', 'LLMs', '. ', 'Split this sentence.']


#### we also need : ? ! as token 

In [ ]:
text="This is new--text ? ?"

In [14]:
result=re.split(r'([,.:;?"_!\']|--|\s)', text)
result=[item for item in result if item.strip()]
result

['This', 'is', 'new', '--', 'text', '?', '?']

#### applying this on our data

In [19]:
preprocessed=re.split(r'([,.:;?"_!\']|--|\s)', raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [20]:
print(len(preprocessed))

4685


## Step 2 creating Token IDs

#### we sort the token to determine the vocabulary size

In [21]:
all_words=sorted(set(preprocessed))
vocab_size=len(all_words)

print(vocab_size)

1131


In [22]:
vocab={token:integer for integer,token in enumerate(all_words)}

In [23]:
for i , item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(I', 3)
('(Though', 4)
(')', 5)
(',', 6)
('--', 7)
('.', 8)
(':', 9)
(';', 10)
('?', 11)
('A', 12)
('Ah', 13)
('Among', 14)
('And', 15)
('Are', 16)
('Arrt', 17)
('As', 18)
('At', 19)
('Be', 20)
('Begin', 21)
('Burlington', 22)
('But', 23)
('By', 24)
('Carlo', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Croft)', 30)
('Destroyed', 31)
('Devonshire', 32)
('Don', 33)
('Dubarry', 34)
('Emperors', 35)
('Florence', 36)
('For', 37)
('Gallery', 38)
('Gideon', 39)
('Gisburn', 40)
('Gisburns', 41)
('Grafton', 42)
('Greek', 43)
('Grindle', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)


#### based on output above the dictionary contain s individual token associated with unique integer labels

In [36]:
class SimpleTokenizerV1:

    def __init__(self, vocab):
        self.str_to_int =vocab
        self.int_to_str={i:s for s,i in vocab.items()}


    def encode(self ,text):
        preprocessed=re.split(r'([,.:;?"_!\']|--|\s)',text)

        preprocessed= [item.strip() for item in preprocessed if item.strip()]

        ids=[self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):

        text=" ".join( self.int_to_str[i] for i in ids)

        text=re.sub(r'([,.:;?"_!\']|--|\s)',r'\1' ,text)
        return text

In [37]:
tokenizer=SimpleTokenizerV1(vocab)

text1=""""It's the last he painted, you know," Mrs. Gisburn said with pardonable pride"""

ids=tokenizer.encode(text1)
print(ids)

[1, 58, 2, 851, 989, 603, 534, 747, 6, 1127, 597, 6, 1, 69, 8, 40, 852, 1109, 755, 794]


#### decoding

In [38]:
decoded=tokenizer.decode(ids)

In [39]:
decoded

'" It \' s the last he painted , you know , " Mrs . Gisburn said with pardonable pride'

### Problem it will give error is something that is not is vocab is give for encoding 

-------------------------------------------------------

## Adding special Context Tokens 

#### we modified SimpleTokenizer to support two new token unknown <|unk|>    and newtext 

In [48]:
len(vocab.items())

1133

In [44]:
all_token=sorted(list(set(preprocessed)))

all_token.extend(["<|endoftext|>"  ,  "<|unk|>"])

vocab={token:integer for integer ,token in enumerate(all_token)}

##### adding endoftext is tell that some data has end new data has started

In [ ]:
#### 

In [49]:
len(vocab.items())

1133

In [45]:
class SimpleTokenizerV2:

    def __init__(self, vocab):
        self.str_to_int =vocab
        self.int_to_str={i:s for s,i in vocab.items()}


    def encode(self ,text):
        preprocessed=re.split(r'([,.:;?"_!\']|--|\s)',text)

        preprocessed= [item.strip() for item in preprocessed if item.strip()]

        preprocessed=[item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

        ids=[self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):

        text=" ".join( self.int_to_str[i] for i in ids)

        text=re.sub(r'([,.:;?"_!\']|--|\s)',r'\1' ,text)
        return text

In [47]:
tokenizer2=SimpleTokenizerV2(vocab)

text1="Hello is not in vocab <|endoftext|>"

tokenizer2.encode(text1)

[1132, 585, 712, 569, 1132, 1131]

In [50]:
for i ,item in enumerate (list(vocab.items())[-5:]):
    print(item)

('younger', 1128)
('your', 1129)
('yourself', 1130)
('<|endoftext|>', 1131)
('<|unk|>', 1132)


#### there are also other special token like

#### [BOS](beginning of sequence):This toke marks the start of text 

#### [EOS](end of sequence):This token marks the end of text useful the concatenating multiple unrelated text

#### [PAD] (padding): batch might contain text of varing length.to ensure the shorter text are extended using padding to make same length

#### GPT USES ONLY ENDOFTEXT AND NOT ANY OF THE OTHER nor unk


#### GPT Uses Byte pair Encoding

# Byte Pair Encoding

#### implementing BPE from scratch is hard we will use open source lib tiktoken for this

In [1]:
!pip install tiktoken

   ---------------------------------------- 0.0/879.4 kB ? eta -:--:--
   ---------------------------------------- 10.2/879.4 kB ? eta -:--:--
   -- ------------------------------------ 61.4/879.4 kB 825.8 kB/s eta 0:00:01
   -------------------- ------------------- 450.6/879.4 kB 4.7 MB/s eta 0:00:01
   -------------------------- ------------- 583.7/879.4 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------  870.4/879.4 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 879.4/879.4 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import importlib
import tiktoken

In [3]:
tokenizer=tiktoken.get_encoding("gpt2")

In [8]:
text=("Tiktoken does thing similar to simpletokenizerv2 we did and even better the it using BPE")

it also deals with endoftext and OOV problem 

In [5]:
integers=tokenizer.encode(text)

In [7]:
integers

[51,
 1134,
 30001,
 857,
 1517,
 2092,
 284,
 2829,
 30001,
 528,
 712,
 17,
 356,
 750,
 290,
 772,
 1365,
 262,
 340]

In [9]:
origialtext=tokenizer.decode(integers)

In [10]:
origialtext

'Tiktoken does thing similar to simpletokenizerv2 we did and even better the it'